# CUDA kernel generation
In this notebook we will explore how to use the `cudakernel` module to generate compute kernels that execute on NVIDIA CUDA-enabled GPUs.

## Construction of a symbolic expression
As our starting point, we need to build a symbolic expression that will be converted into a compute kernel. In this example, we construct the 2D divergence of a vector field $F(u,v) = [u, v]$. For now, the divergence will be generate as a single expression. Later on, we will show how to define variables that can hold intermediate results of the computation. 

In [1]:
import openfd as fd
from sympy import symbols
n = symbols('n')
u = fd.GridFunction('u', shape=(n, n))
v = fd.GridFunction('v', shape=(n, n))
out = fd.GridFunction('out', shape=(n, n))
u_x = fd.sbp_traditional.Derivative(u, 'x', order=4, gpu=True)
v_y = fd.sbp_traditional.Derivative(v, 'y', order=4, gpu=True)
div = fd.GridFunctionExpression(u_x + v_y)
div


Dx(u) + Dy(v)

In the construction of the grid functions, it is important to specify `gpu=True`. This parameter forces the derivative to store its data in arrays. To see this in action, we can evaluate the expression `div` somewhere.
The name of the array can be changed by addition to setting `gpu=True` also setting `coef='name'`, where `name` can be any valid variable name.

In [2]:
div[0,0]

dxl[i][0]*u[0, 0] + dxl[i][1]*u[1, 0] + dxl[i][2]*u[2, 0] + dxl[i][3]*u[3, 0] + dxl[i][4]*u[4, 0] + dxl[i][5]*u[5, 0] + dyl[j][0]*v[0, 0] + dyl[j][1]*v[0, 1] + dyl[j][2]*v[0, 2] + dyl[j][3]*v[0, 3] + dyl[j][4]*v[0, 4] + dyl[j][5]*v[0, 5]

To obtain the array values needed for the computation, we select the compute region wish to obtain data for. There are three compute regions, corresponding to the left boundary, interior, and right boundary. 

In [3]:
# left boundary
u_x.coef(0)

('dxl',
 array([[-1.41176471,  1.73529412, -0.23529412, -0.08823529,  0.        ,
          0.        ],
        [-0.5       ,  0.        ,  0.5       ,  0.        ,  0.        ,
          0.        ],
        [ 0.09302326, -0.68604651,  0.        ,  0.68604651, -0.09302326,
          0.        ],
        [ 0.03061224,  0.        , -0.60204082,  0.        ,  0.65306122,
         -0.08163265]]))

In [4]:
# interior
u_x.coef(1)

('dx', array([ 0.08333333, -0.66666667,  0.66666667, -0.08333333]))

In [5]:
# right boundary
u_x.coef(-1)

('dxr',
 array([[ 1.41176471, -1.73529412,  0.23529412,  0.08823529, -0.        ,
         -0.        ],
        [ 0.5       , -0.        , -0.5       , -0.        , -0.        ,
         -0.        ],
        [-0.09302326,  0.68604651, -0.        , -0.68604651,  0.09302326,
         -0.        ],
        [-0.03061224, -0.        ,  0.60204082, -0.        , -0.65306122,
          0.08163265]]))

## CUDA kernel generation
The next step is to familiarize ourselves with the CUDA module. We will use the function `kernel2d` that builds 2D compute kernels. This function generates code for different parts of the computational domain. 

In 2D the computational domain (a rectangular grid) is split into 9 different regions. These regions correspond to the fact that in 1D there are three different regions: left boundary, interior, and right boundary as previously explained. For now, let us focus on the interior only.

In [6]:
region = (1, 1)

Another piece of information we need to provide is where each compute region begins and ends. This information is conveniently provided by the derivatives 

In [7]:
bounds = (u_x.bounds(), v_y.bounds())

Sometimes, the bounds from the derivative are not the appropriate ones to use. Fortunately, it is fairly straight forward to define custom ones as follows

In [8]:
custombounds = (fd.Bounds(size=10, left=1, right=1), fd.Bounds(size=10, left=1, right=1))
# give index range for the left boundary in the x-direction
custombounds[0].range(0)

(0, 1)

In [9]:
# ... interior
custombounds[0].range(1)

(1, 9)

In [10]:
# ... and right boundary
custombounds[0].range(-1)

(9, 10)

It is also necessary to pass the array data to the kernel generator.

In [11]:
arrays = []
arrays.append(u_x.coef(region[0]))
arrays.append(v_y.coef(region[1]))
arrays

[('dx', array([ 0.08333333, -0.66666667,  0.66666667, -0.08333333])),
 ('dy', array([ 0.08333333, -0.66666667,  0.66666667, -0.08333333]))]

**developer notes** Note that these arrays contain the same data. It should be sufficient to only have one array in this case.

Once we have all this information, we can go ahead and generate the CUDA code to compute the 2D divergence in the interior of the domain. We will assign the result of the computation to the grid function `out`, constructed earlier.

In [12]:
from openfd.dev.cudakernel import cudakernel, kernel2d 
code = kernel2d(bounds, region, arrays, out, div)
print(code)

const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= n - 8) return;
if ( j >= n - 8) return;
const float dx[4] = {0.0833333, -0.666667, 0.666667, -0.0833333};
const float dy[4] = {0.0833333, -0.666667, 0.666667, -0.0833333};
out[i + n*(j + 4) + 4] = dx[0]*u[i + n*(j + 4) + 2] + dx[1]*u[i + n*(j + 4) + 3] + dx[2]*u[i + n*(j + 4) + 5] + dx[3]*u[i + n*(j + 4) + 6] + dy[0]*v[i + n*(j + 2) + 4] + dy[1]*v[i + n*(j + 3) + 4] + dy[2]*v[i + n*(j + 5) + 4] + dy[3]*v[i + n*(j + 6) + 4];



Let us also see what the computation looks like on the boundary. In this case, the coefficients are stored in a two dimensional array where each row contains the coefficients for the given boundary point.

In [13]:
region = (0, 0)
arrays = []
arrays.append(u_x.coef(region[0]))
arrays.append(v_y.coef(region[1]))
code = kernel2d(bounds, region, arrays, out, div)
print(code)

const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= 4) return;
if ( j >= 4) return;
const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
out[i + j*n] = dxl[i][0]*u[j*n] + dxl[i][1]*u[j*n + 1] + dxl[i][2]*u[j*n + 2] + dxl[i][3]*u[j*n + 3] + dxl[i][4]*u[j*n + 4] + dxl[i][5]*u[j*n + 5] + dyl[j][0]*v[i] + dyl[j][1]*v[i + n] + dyl[j][2]*v[i + 2*n] + dyl[j][3]*v[i + 3*n] + dyl[j][4]*v[i + 4*n] + dyl[j][5]*v[i + 5*n];



Next, we will generate a function header for the block of code shown above. The kernel function generator will automatically detect which parameters are in and output arguments, and also determine their type. There are some limitations at the moment. Any symbols nested inside an operator will not be detected. For the detection to work, we will need to pass the left-hand and right-hand side expressions as before. 

In [14]:
code = cudakernel('divergence_00', n, out, div, code)
print(code)

__global__ void divergence_00(float *out, const float *u, const float *v, const int n) {
         const int i = threadIdx.x + blockIdx.x*blockDim.x;
         const int j = threadIdx.y + blockIdx.y*blockDim.y;
         if ( i >= 4) return;
         if ( j >= 4) return;
         const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
         const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
         out[i + j*n] = dxl[i][0]*u[j*n] + dxl[i][1]*u[j*n + 1] + dxl[i][2]*u[j*n + 2] + dxl[i][3]*u[j*n + 3] + dxl[i][4]*u[j*n + 4] + dxl[i][5]*u[j*n + 5] + dyl[j][0]*v[i] + dyl[j][1]*v[i + n] + dyl[j][2]*v[i + 2*n] + dyl[j][3]*v[i + 3*n] + dyl[j][4]*v[i + 4*n] + dyl[j][5]*v[i + 5*n];
         
}




## Multiline expressions and intermediate variables

For readibility, and also in some applications, it is useful to be able to store intermediate results of computations. These intermediate results can then be reused in other parts of the compute kernel(s).

We revisit the divergence example, but this time we will breakup the expression into multiple ones by defining a variable for each derivative. 


In [15]:
from openfd.dev.variable import Variable
# Previous code
u_x = fd.sbp_traditional.Derivative(u, 'x', order=2, gpu=True)
v_y = fd.sbp_traditional.Derivative(v, 'y', order=2, gpu=True)
# Variable declarations
var_x = Variable('u_x', u_x)
var_y = Variable('v_y', v_y)

div = fd.GridFunctionExpression(var_x + var_y)


Now, when `div` is accessed it will no longer print out the complete expression. Instead, `u_x` and `v_y` will be referenced. 

In [16]:
div

u_x + v_y

It is important to note that index accessing now becomes meaningless

In [17]:
div[0,0]

u_x + v_y

The value assigned to a variable can be accessed using

In [18]:
var_x.val

Dx(u)

Finally, we can go ahead and modify the kernel generation call so that we first compute the derivatives and store them in `u_x` and `v_y`, and then assign the final output to `out` by adding these variables together.

In [19]:
code = kernel2d(bounds, region, arrays, [var_x.declare('const float'), var_y.declare('const float'), out], [var_x.val, var_y.val, div])
print(var_x[0,0])
print(code)

u_x
const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= 4) return;
if ( j >= 4) return;
const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float u_x = dxl[i][0]*u[j*n] + dxl[i][1]*u[j*n + 1];
const float v_y = dyl[j][0]*v[i] + dyl[j][1]*v[i + n];
out[i + j*n] = u_x + v_y;



Note that we have to call the `declare` function for each variable and assign a type to it. If we do not declare it, an exception is thrown by the kernel generator. 

## Block sizes and grid sizes
To execute the kernel, we need to provide the number of threads per block to use. The number of blocks to use for the computation will be automatically computed. The number of threads per block depends on in which region the computation takes place in. For the interior region, it is possible to set the number of threads per block to use here. It is also possible to specify the number of threads per block on the boundary. We will use 32 threads (one warp) for the first grid dimension, and similarily for the boundary.

In [20]:
block = (32, 1, 1)
boundary_block = (32, 1, 1)

We also need to provide a numerical value for the grid size (we use the same grid size in each dimension). This size will be used to determine the number of blocks needed for the computation.

In [21]:
n_ = 100
shape = (n_, n_, 1)

In [22]:
from openfd.dev.cudakernel import Grid 
G = Grid(bounds, shape, block=block, boundary_block=boundary_block)

The grid object `G` can then be used to return the number of threads per block and number of blocks for different compute regions. If we ask for more threads than what can be used to perform the computation, the number of threads per block is automatically adjusted to fit.

In [23]:
G.block(0,0)

(4, 1, 1)

In [24]:
G.block(1,0)

(32, 1, 1)

In [25]:
G.block(1,1)

(32, 1, 1)

In [26]:
G.block(0,1)

(4, 1, 1)

In the same way, we can see the number of blocks on the grid. If the number of blocks is not divisible by the grid dimensions, then extra blocks are added to ensure that the entire compute region is covered by blocks.

In [27]:
G.grid(1,1)

(3, 92, 1)

In [28]:
G.grid(1,0)

(3, 4, 1)

In [29]:
G.grid(0,0)

(1, 4, 1)

## Debugging
In many cases, it easy to make a mistake when configuring the bounds for the compute kernels. For this reason, there is a simple debugging feature built-in to the cudakernel generator that writes the region ID for a given compute kernel. The intended usage is to visualize the output array to see which parts of it are accessed by the different compute kernels.

The following example changes the region parameter and we can observe that the output `out` is set to the corresponding region number when `debug=True`.

In [30]:
code = kernel2d(bounds, (0, 0), arrays, out, div, debug=True)
print(code)

const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= 4) return;
if ( j >= 4) return;
const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
out[i + j*n] = 1;



In [31]:
code = kernel2d(bounds, (1, 0), arrays, out, div, debug=True)
print(code)

const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= n - 8) return;
if ( j >= 4) return;
const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
out[j*n + i + 4] = 2;



In [32]:
code = kernel2d(bounds, (-1, 0), arrays, out, div, debug=True)
print(code)

const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= 4) return;
if ( j >= 4) return;
const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
out[j*n - i + n - 1] = 3;



In [33]:
code = kernel2d(bounds, (0, 1), arrays, out, div, debug=True)
print(code)

const int i = threadIdx.x + blockIdx.x*blockDim.x;
const int j = threadIdx.y + blockIdx.y*blockDim.y;
if ( i >= 4) return;
if ( j >= n - 8) return;
const float dxl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
const float dyl[4][6] = {-1.41176, 1.73529, -0.235294, -0.0882353, 0, 0, -0.5, 0, 0.5, 0, 0, 0, 0.0930233, -0.686047, 0, 0.686047, -0.0930233, 0, 0.0306122, 0, -0.602041, 0, 0.653061, -0.0816327};
out[i + n*(j + 4)] = 4;

